In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

/Users/adithyaabhishek/opt/anaconda3/envs/nlpev/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
injury_df = pd.read_pickle("Data/injury_descriptions_processed.pkl")

In [5]:
injury_new_df = injury_df[
    ~injury_df.InjuryCauseDesc.str.contains("Bodily reaction and exertion, unspecified")]
injuryDataFrame = injury_df[~injury_df.InjuryCauseDesc.str.contains("Bodily reaction, n.e.c.")]

injury_new_df.reset_index(inplace=True)
injury_new_df.drop(['index','label'], axis=1,inplace=True)

le = LabelEncoder()
label = le.fit_transform(injury_new_df.InjuryCauseDesc)
injury_new_df['label']=label


/var/folders/gy/hjflsmyx53jch_h0701bpwdc0000gn/T/ipykernel_63336/166171370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injury_new_df.drop(['index','label'], axis=1,inplace=True)
/var/folders/gy/hjflsmyx53jch_h0701bpwdc0000gn/T/ipykernel_63336/166171370.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injury_new_df['label']=label


In [6]:
injury_new_df


,InjuryCauseDesc,InjuryDesc,label
0,Struck by falling object,security pier painting whale mural keeping are...,10
1,"Bending, climbing, crawling, reaching, twisting",iw moving carton shaving cream sudden onset lbp,0
2,"Bending, climbing, crawling, reaching, twisting",picking box file hurt lower back,0
3,Struck by falling object,beam slipped struck ee right wrist,10
4,Overexertion in lifting,employee stated felt strain lower back groin l...,4
...,...,...,...
94435,Struck by slipping handheld object,employee slicing food knife slipped lacerated ...,11
94436,Rubbed or abraded by foreign matter in eye,employee cleaning davinci arm using psi liquid...,7
94437,"Bending, climbing, crawling, reaching, twisting",allegedly tweaked back checking cavallaro order,0
94438,"Fall to floor, walkway, or other surface",employee report sitting chair went get leaned ...,3


In [7]:
def token_process_split(data_local):
    vectorizer = TfidfVectorizer(max_features=4000, decode_error="ignore",ngram_range=(1,2))
    Xvec = vectorizer.fit_transform(data_local.InjuryDesc)
    y = data_local.label
    
    x_train,x_test,y_train,y_test = train_test_split(Xvec,y,test_size=0.2,random_state=42)
    
    return x_train,x_test,y_train,y_test

# whole dataset

In [10]:
x_train_w,x_test_w,y_train_w,y_test_w = token_process_split(injury_new_df)
x_train_w,x_test_w,y_train_w,y_test_w

(<75552x4000 sparse matrix of type '<class 'numpy.float64'>'
 	with 1055770 stored elements in Compressed Sparse Row format>,
 <18888x4000 sparse matrix of type '<class 'numpy.float64'>'
 	with 262210 stored elements in Compressed Sparse Row format>,
 86020     9
 1409      8
 88986     3
 56629     4
 36873     3
          ..
 6265      0
 54886    11
 76820     2
 860       2
 15795     8
 Name: label, Length: 75552, dtype: int64,
 78450    8
 34191    0
 58492    2
 60737    2
 83327    3
         ..
 30661    3
 58290    6
 55066    2
 80358    6
 21338    3
 Name: label, Length: 18888, dtype: int64)

In [14]:
from xgboost import XGBClassifier

In [34]:
xgb_model = XGBClassifier()
xgb_model.fit(x_train_w,y_train_w)

/Users/adithyaabhishek/opt/anaconda3/envs/nlpev/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:06:35] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [35]:
y_pred = xgb_model.predict(x_test_w)

In [36]:
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test_w, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.25%


In [37]:
y_pred_train = xgb_model.predict(x_train_w)

predictions_train = [round(value) for value in y_pred_train]

accuracy = accuracy_score(y_train_w, predictions_train)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.50%


# brachet 2 - 6 classes

In [19]:
# splitting classes for experimentation
data = injury_new_df
X0 = data[data.label == 0]
X1 = data[data.label == 1]
X2 = data[data.label == 2]
X3 = data[data.label == 3]
X4 = data[data.label == 4]
X5 = data[data.label == 5]
X6 = data[data.label == 6]
X7 = data[data.label == 7]
X8 = data[data.label == 8]
X9 = data[data.label == 9]
X10 = data[data.label == 10]
X11 = data[data.label == 11]

X3_ds = X3.sample(n=12000)
X4_ds = X4.sample(n=12000)
X2_ds = X2.sample(n=12000)

x8_s = X8.sample(n=(10000-len(X9)))
x5_s = X5.sample(n=(8000-len(X1)))
x10_s = X10.sample(n=(8000-len(X6)))



In [20]:
data_lrg2 = pd.concat([X3_ds,X4_ds,X2_ds,X8,X5,X10,x8_s,x5_s,x10_s])

In [21]:
x_train2,x_test2,y_train2,y_test2 = token_process_split(data_lrg2)

In [22]:
xgb_model.fit(x_train2,y_train2)

/Users/adithyaabhishek/opt/anaconda3/envs/nlpev/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:59:28] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [24]:
y_pred = xgb_model.predict(x_test2)

predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test2, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 82.41%


In [25]:
y_pred_train = xgb_model.predict(x_train2)

predictions_train = [round(value) for value in y_pred_train]

accuracy = accuracy_score(y_train2, predictions_train)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.22%


# Top 3 classes

In [26]:
data_lrg = pd.concat([X3,X4,X5])

In [27]:
x_train,x_test,y_train,y_test = token_process_split(data_lrg)
x_train,x_test,y_train,y_test

(<29172x4000 sparse matrix of type '<class 'numpy.float64'>'
 	with 441232 stored elements in Compressed Sparse Row format>,
 <7294x4000 sparse matrix of type '<class 'numpy.float64'>'
 	with 110863 stored elements in Compressed Sparse Row format>,
 80493    4
 49770    4
 72033    5
 72965    3
 41598    3
         ..
 4118     4
 46648    3
 74996    3
 8090     3
 93972    3
 Name: label, Length: 29172, dtype: int64,
 10465    4
 22854    3
 73965    5
 20786    4
 71637    5
         ..
 69891    4
 17810    3
 61853    3
 57309    5
 39516    3
 Name: label, Length: 7294, dtype: int64)

In [28]:
xgb_model.fit(x_train,y_train)

/Users/adithyaabhishek/opt/anaconda3/envs/nlpev/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:03:30] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [32]:
y_pred = xgb_model.predict(x_test)

predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.53%


In [33]:
y_pred_train = xgb_model.predict(x_train)

predictions_train = [round(value) for value in y_pred_train]

accuracy = accuracy_score(y_train, predictions_train)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 96.06%
